<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

Get the settings from the settings.yaml file.

Get the data from the Azure SQL database in a pandas DataFrame or from a previously stored pickle file.

For every project in the peilbuizen dataframe, the shortest route is calculated and saved to an Excel file and a URL file.

In [0]:
#| echo: false
#| output: asis
show_doc(create_group_route)

---

### create_group_route

>      create_group_route (start_address:str,
>                          group_df:pandas.core.frame.DataFrame,
>                          route_profile:str, project_name:str,
>                          output_dir:str='output', current_date:str=None)

*Create optimized route for a group and save results*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| start_address | str |  | De startlocatie en eindlocatie opgegeven als adres |
| group_df | DataFrame |  | De pandas dataframe met peilbuizen in de groep, bijv. op basis van project |
| route_profile | str |  | Het route profiel (transport methode) waarvoor de route berekend moet worden |
| project_name | str |  | Naam van de groep, bijv. Project Vecht |
| output_dir | str | output | Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder. |
| current_date | str | None | De datum. Als niks wordt opgegeven, wordt de huidige datum gebruikt. |
| **Returns** | **None** |  |  |

In [0]:
#| echo: false
#| output: asis
show_doc(process_peilbuizen_routes)

---

### process_peilbuizen_routes

>      process_peilbuizen_routes (df:pandas.core.frame.DataFrame,
>                                 start_address:str, route_profile:str,
>                                 output_dir:str='output')

*Process peilbuizen dataframe, create optimized routes for each group,
and save results to Excel and URL files

Args:
    df: DataFrame with peilbuizen data
    start_address: Starting location address
    route_profile: De route profiel (transport methode) waarvoor de route berekend moet worden
    output_dir: Directory to save output files*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DataFrame |  | Dataframe van alle peilbuis groepen waarvoor route berekend wordt |
| start_address | str |  | De startlocatie en eindlocatie opgegeven als adres |
| route_profile | str |  | De route profiel (transport methode) waarvoor de route berekend moet worden |
| output_dir | str | output | Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder. |
| **Returns** | **None** |  |  |